In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_value, get_optimal_action, get_optimal_actions, get_model_actions, test_model, one_batch, test_model_accuracy, one_batch_supervised, train, train_supervised, create_model

# 单步奖励收集

这是一个用于练习策略梯度法 (policy gradient method) 的玩具题。  
8个格子首尾相连，其中随机3个格子奖励，奖励数值为0~1的均匀随机数。  
动作：选择一个格子。
计分规则：  
1. 如果格子有奖励（大于0），获得对应的奖励。
2. 如果格子是空的（等于0），且两侧的格子都有奖励，会获得两侧格子的总奖励。
3. 否则奖励为0。

## 测试监督学习效果

In [2]:
for optimilizer in ['sgd', 'adam']:
    for n_hidden_layers in [1, 2]:
        for n_dense_units in [64, 128, 256, 512]:
            for ratio_dropout in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
                model_supervised = create_model(n_hidden_layers, n_dense_units, ratio_dropout, optimilizer)
                scores = np.array([one_batch_supervised(model_supervised, batch_size=128, n_test_rounds=10000) for i in np.arange(300)])
                argmax = scores.argmax()
                print(optimilizer, n_hidden_layers, n_dense_units, ratio_dropout, argmax, scores[argmax])

sgd 1 64 0.2 290 0.8894605271764167
sgd 1 64 0.3 295 0.8761614984673568
sgd 1 64 0.4 289 0.8733202250553918
sgd 1 64 0.5 292 0.877693469215966
sgd 1 64 0.6 269 0.8733585605920671
sgd 1 64 0.7 249 0.8671943778947623
sgd 1 128 0.2 181 0.8969206804154661
sgd 1 128 0.3 180 0.8960513484973998
sgd 1 128 0.4 293 0.8957055927253046
sgd 1 128 0.5 276 0.8953302068304051
sgd 1 128 0.6 295 0.8924188652748765
sgd 1 128 0.7 253 0.8941561621538368
sgd 1 256 0.2 278 0.904177163572226
sgd 1 256 0.3 221 0.9040284736918341
sgd 1 256 0.4 260 0.9062705495465576
sgd 1 256 0.5 253 0.903307252891115
sgd 1 256 0.6 259 0.8997506592172958
sgd 1 256 0.7 253 0.8993784495272655
sgd 1 512 0.2 259 0.9131638545916596
sgd 1 512 0.3 276 0.9097489778531546
sgd 1 512 0.4 225 0.9079081757583483
sgd 1 512 0.5 253 0.9111476706843749
sgd 1 512 0.6 280 0.9075796399391469
sgd 1 512 0.7 230 0.9063435648831448
sgd 2 64 0.2 293 0.8947735987399619
sgd 2 64 0.3 251 0.8798183790991799
sgd 2 64 0.4 293 0.8822541154401481
sgd 2 64 0.5 